# 📦 JSON & Arrays

This notebook covers:
- JSON functions (PARSE_JSON, OBJECT_CONSTRUCT, GET_PATH)
- Array functions (ARRAY_CONSTRUCT, ARRAY_SIZE, FLATTEN)
- Working with semi-structured data

In [1]:
import snowflake.connector

from snowduck import start_patch_snowflake

start_patch_snowflake(db_file="examples.duckdb", reset=True)

## JSON Functions

In [2]:
# PARSE_JSON - Parse JSON string
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT PARSE_JSON('{"user": "Alice", "score": 95, "active": true}') as data
    """)
    print(f"Parsed JSON: {cursor.fetchone()[0]}")

Parsed JSON: {"user": "Alice", "score": 95, "active": true}


In [3]:
# OBJECT_CONSTRUCT - Build JSON from key-value pairs
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT OBJECT_CONSTRUCT('name', 'Bob', 'age', 30, 'city', 'NYC') as user_obj
    """)
    print(f"Constructed: {cursor.fetchone()[0]}")

Constructed: {"name":"Bob","age":30,"city":"NYC"}


In [4]:
# GET_PATH - Extract values from JSON
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT GET_PATH(
            PARSE_JSON('{"user": {"name": "Alice", "id": 123}}'),
            'user.name'
        ) as name
    """)
    print(f"Extracted name: {cursor.fetchone()[0]}")

Extracted name: Alice


In [5]:
# JSON in table columns
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("CREATE DATABASE IF NOT EXISTS demo")
    cursor.execute("USE DATABASE demo")

    cursor.execute("""
        CREATE OR REPLACE TABLE user_profiles (
            id INT,
            profile VARCHAR
        )
    """)

    cursor.execute("""
        INSERT INTO user_profiles VALUES
        (1, '{"name": "Alice", "skills": ["Python", "SQL"]}'),
        (2, '{"name": "Bob", "skills": ["JavaScript", "React"]}'),
        (3, '{"name": "Carol", "skills": ["Python", "ML"]}')
    """)

    cursor.execute("""
        SELECT 
            id,
            GET_PATH(PARSE_JSON(profile), 'name') as name
        FROM user_profiles
        ORDER BY id
    """)

    print("Users:")
    for row in cursor.fetchall():
        print(f"  ID {row[0]}: {row[1]}")

Users:
  ID 1: Alice
  ID 2: Bob
  ID 3: Carol


## Array Functions

In [6]:
# ARRAY_CONSTRUCT - Create arrays
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            ARRAY_CONSTRUCT(1, 2, 3, 4, 5) as numbers,
            ARRAY_CONSTRUCT('apple', 'banana', 'cherry') as fruits
    """)
    row = cursor.fetchone()
    print(f"Numbers: {row[0]}")
    print(f"Fruits: {row[1]}")

Numbers: [1, 2, 3, 4, 5]
Fruits: ['apple', 'banana', 'cherry']


In [7]:
# ARRAY_SIZE, ARRAY_CONTAINS, ARRAY_TO_STRING
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            ARRAY_SIZE(ARRAY_CONSTRUCT(1, 2, 3)) as size,
            ARRAY_CONTAINS(2, ARRAY_CONSTRUCT(1, 2, 3)) as contains_2,
            ARRAY_TO_STRING(ARRAY_CONSTRUCT('a', 'b', 'c'), ',') as joined
    """)
    row = cursor.fetchone()
    print(f"Size: {row[0]}")
    print(f"Contains 2: {row[1]}")
    print(f"Joined: {row[2]}")

Size: 3
Contains 2: True
Joined: a,b,c


In [8]:
# ARRAY_DISTINCT, ARRAY_INTERSECTION
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            ARRAY_DISTINCT(ARRAY_CONSTRUCT(1, 2, 2, 3, 3, 3)) as distinct_nums,
            ARRAY_INTERSECTION(
                ARRAY_CONSTRUCT(1, 2, 3, 4),
                ARRAY_CONSTRUCT(3, 4, 5, 6)
            ) as common
    """)
    row = cursor.fetchone()
    print(f"Distinct: {row[0]}")
    print(f"Intersection: {row[1]}")

Distinct: [3, 2, 1]
Intersection: [4, 3]


In [9]:
# ARRAY_SLICE
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT 
            ARRAY_SLICE(ARRAY_CONSTRUCT(1, 2, 3, 4, 5), 1, 3) as sliced
    """)
    print(f"Slice [1:3]: {cursor.fetchone()[0]}")

Slice [1:3]: [2, 3, 4]


## FLATTEN - Explode Arrays

In [10]:
# Basic FLATTEN
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT value
        FROM TABLE(FLATTEN(INPUT => ARRAY_CONSTRUCT(10, 20, 30, 40, 50)))
        ORDER BY value
    """)
    print("Flattened values:")
    for row in cursor.fetchall():
        print(f"  {row[0]}")

Flattened values:
  10
  20
  30
  40
  50


In [11]:
# FLATTEN with string array
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT value
        FROM TABLE(FLATTEN(INPUT => ARRAY_CONSTRUCT('apple', 'banana', 'cherry')))
        ORDER BY value
    """)
    print("Fruits:")
    for row in cursor.fetchall():
        print(f"  {row[0]}")

Fruits:
  apple
  banana
  cherry


## Arrays in Tables

In [12]:
# Create table with ARRAY column
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        CREATE OR REPLACE TABLE products (
            id INT,
            name VARCHAR,
            tags ARRAY
        )
    """)

    cursor.execute("""
        INSERT INTO products VALUES 
            (1, 'Laptop', [101, 102, 103]),
            (2, 'Phone', [102, 104]),
            (3, 'Tablet', [101, 103, 105])
    """)
    print("Created products table")

Created products table


In [13]:
# Query with ARRAY_CONTAINS
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT id, name
        FROM products
        WHERE ARRAY_CONTAINS(102, tags)
        ORDER BY id
    """)
    print("Products with tag 102:")
    for row in cursor.fetchall():
        print(f"  {row[0]}: {row[1]}")

Products with tag 102:
  1: Laptop
  2: Phone


In [14]:
# Query with ARRAY_SIZE
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        SELECT name, ARRAY_SIZE(tags) as num_tags
        FROM products
        ORDER BY num_tags DESC
    """)
    print("Products by tag count:")
    for row in cursor.fetchall():
        print(f"  {row[0]}: {row[1]} tags")

Products by tag count:
  Laptop: 3 tags
  Tablet: 3 tags
  Phone: 2 tags


## ARRAY_AGG - Aggregate into Arrays

In [15]:
with snowflake.connector.connect() as conn, conn.cursor() as cursor:
    cursor.execute("""
        CREATE OR REPLACE TABLE items (
            category VARCHAR,
            item VARCHAR
        )
    """)
    cursor.execute("""
        INSERT INTO items VALUES
            ('fruit', 'apple'),
            ('fruit', 'banana'),
            ('fruit', 'orange'),
            ('veg', 'carrot'),
            ('veg', 'broccoli')
    """)

    cursor.execute("""
        SELECT 
            category,
            ARRAY_AGG(item) as items
        FROM items
        GROUP BY category
        ORDER BY category
    """)

    print("Items by category:")
    for row in cursor.fetchall():
        print(f"  {row[0]}: {row[1]}")

Items by category:
  fruit: ['apple', 'banana', 'orange']
  veg: ['carrot', 'broccoli']
